<a href="https://colab.research.google.com/github/cunhamaicon/petr4/blob/master/petr4_rnn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importação dos pacotes ncessários:

In [127]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from keras.layers import LSTM
from tensorflow.keras.optimizers import SGD
import keras
from functions import *
import keras.backend as K

In [86]:
print(tf.__version__)

2.6.0


In [2]:
TICKS = 20
N_INPUTS = 5
COLUMN_NAME = 'Rate Close'

###Criando os bancos:

In [3]:
dataset = pd.read_csv(r"history_datas\AAPL_1d.csv", sep='\t')
dataset

,Date,Open,High,Low,Close,Volume
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,493729600
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,601904800
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,552160000
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,477131200
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,447610800
...,...,...,...,...,...,...
2509,2019-12-20,70.557503,70.662498,69.639999,69.860001,275978000
2510,2019-12-23,70.132500,71.062500,70.092499,71.000000,98572000
2511,2019-12-24,71.172501,71.222504,70.730003,71.067497,48478800
2512,2019-12-26,71.205002,72.495003,71.175003,72.477501,93121200


In [4]:
def creat_rate_close(dataset, column_name):
    rate_close = [0.001]
    for close in range(1, len(dataset[column_name])):
        rc = (dataset[column_name][close] - dataset[column_name][close-1])/dataset[column_name][close-1]
        rate_close.append(rc)
    dataset[COLUMN_NAME] = rate_close
    return dataset

def fit_dados(df:object, df_y:list, tick_treino:int):
    X_train = []
    y_train = []
    dados_ptreino = len(df)
    for i in range(tick_treino, dados_ptreino-1):
        temp = []
        for c in range(i-tick_treino,i):
            temp.append(df[c:c+1].values.tolist()[0])
        y_train.append(df_y[i])
        X_train.append(temp)

    X_train, y_train = np.array(X_train), np.array(y_train)
    
    return X_train,y_train


In [5]:
dataset = creat_rate_close(dataset, 'Close')

In [6]:
dataset

,Date,Open,High,Low,Close,Volume,Rate Close
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,493729600,0.001000
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,601904800,0.001729
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,552160000,-0.015906
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,477131200,-0.001849
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,447610800,0.006648
...,...,...,...,...,...,...,...
2509,2019-12-20,70.557503,70.662498,69.639999,69.860001,275978000,-0.002071
2510,2019-12-23,70.132500,71.062500,70.092499,71.000000,98572000,0.016318
2511,2019-12-24,71.172501,71.222504,70.730003,71.067497,48478800,0.000951
2512,2019-12-26,71.205002,72.495003,71.175003,72.477501,93121200,0.019840


In [7]:
dataset.drop(columns=['Date'], inplace=True)
dataset.drop(columns=['Close'], inplace=True)

In [8]:
dataset

,Open,High,Low,Volume,Rate Close
0,7.622500,7.660714,7.585000,493729600,0.001000
1,7.664286,7.699643,7.616071,601904800,0.001729
2,7.656429,7.686786,7.526786,552160000,-0.015906
3,7.562500,7.571429,7.466071,477131200,-0.001849
4,7.510714,7.571429,7.466429,447610800,0.006648
...,...,...,...,...,...
2509,70.557503,70.662498,69.639999,275978000,-0.002071
2510,70.132500,71.062500,70.092499,98572000,0.016318
2511,71.172501,71.222504,70.730003,48478800,0.000951
2512,71.205002,72.495003,71.175003,93121200,0.019840


In [9]:
#CONTANTO QUANTO A FREQUENCIA DE CRESCIMENTO E DE QUEDA DA AÇÃO
(dataset[COLUMN_NAME]>0).value_counts()

True     1328
False    1186
Name: Rate Close, dtype: int64

In [23]:
X, Y = fit_dados(dataset, dataset[COLUMN_NAME].values, TICKS)

In [24]:
#Check consistencia na formatação dos dados
for c in range(len(Y)):
    if not dataset[COLUMN_NAME][c+TICKS]==Y[c]:
        print(f'Erro na linha {c}')
    #pd.DataFrame(X_train[i]).values == list(dataset[i:i+20].values

In [12]:
sc = MinMaxScaler(feature_range = (0, 1))

Y = sc.fit_transform(np.reshape(np.array(Y), (len(Y),1)))

**Creating Loss function

In [25]:
y_true=np.array([[1.0]])
y_pred = np.array([[0.9]])

loss = keras.losses.BinaryCrossentropy()
loss(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=0.10536040365695953>

In [110]:
aqui
from tensorflow.python.ops import math_ops


In [78]:
def custom_loss_1(y_true, y_pred):
    def check_dirc(y_t, y_p):
        return True if y_t*y_p>0 else False

    penal = 0.20
    tensor = []
    for y_t, y_p  in zip(y_true[0].eval(), y_pred[0].eval()):
        value = abs(y_t - y_p)*80
        if check_dirc(y_t, y_p):
            value += penal
        tensor.append(value)
    return tensor

In [124]:
aqui.teste.__dir__()

['_op',
 '_value_index',
 '_dtype',
 '_tf_output',
 '_shape_val',
 '_consumers',
 '_id',
 '_name',
 '__module__',
 '__doc__',
 'OVERLOADABLE_OPERATORS',
 '_USE_EQUALITY',
 '__init__',
 '__getattr__',
 '_create_with_tf_output',
 'op',
 'dtype',
 'graph',
 'name',
 'device',
 'shape',
 '_c_api_shape',
 '_shape',
 '_disallow_when_autograph_disabled',
 '_disallow_when_autograph_enabled',
 '_disallow_in_graph_mode',
 '_disallow_bool_casting',
 '_disallow_iteration',
 '__iter__',
 '_shape_as_list',
 '_shape_tuple',
 '_rank',
 'get_shape',
 'set_shape',
 'value_index',
 'consumers',
 '_as_node_def_input',
 '_as_tf_output',
 '__str__',
 '__repr__',
 '__hash__',
 '__copy__',
 '__array_priority__',
 '__array__',
 '__len__',
 '_override_operator',
 '__bool__',
 '__nonzero__',
 'eval',
 'experimental_ref',
 'ref',
 '_tf_api_names',
 '_tf_api_names_v1',
 '__getitem__',
 '__neg__',
 '__abs__',
 '__add__',
 '__radd__',
 '__sub__',
 '__rsub__',
 '__mul__',
 '__rmul__',
 '__div__',
 '__rdiv__',
 '__tru

In [112]:
def custom_loss(y_true, y_pred):
    diff = math_ops.squared_difference(y_pred, y_true)  #squared difference
    aqui.teste= diff
    return loss

In [113]:
a_tensor = tf.constant([[1, 2, 3], [4, 5, 6]])
a_tensor.numpy()

array([[1, 2, 3],
       [4, 5, 6]])

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,  test_size=0.3, shuffle=False)

In [115]:
def model_LSTM():
  regressor = Sequential()

  regressor.add(LSTM(units = 33, activation='tanh',  return_sequences = True, input_shape = (TICKS, N_INPUTS)))

  
  regressor.add(LSTM(units = 33,return_sequences = True, kernel_regularizer = regularizers.l2 (0.0001)))
 
  
  regressor.add(LSTM(units = 33, return_sequences = True))
 

  regressor.add(LSTM(units = 33))
 

  regressor.add(Dense(1, activation='tanh'))

  opt = SGD(lr=0.01, momentum=0.9)

  regressor.compile(optimizer='rmsprop', loss=custom_loss)
  
  return regressor

## Construindo a RNN

In [116]:
def model_DMLP():
  model = Sequential()

  model.add(Dense(2, activation="tanh", input_shape=(TICKS, N_INPUTS)))
  model.add(Dense(10))
  model.add(Dense(10))
  model.add(Dense(1, activation="tanh"))

  model.compile(optimizer = "adam", loss='mean_absolute_error')
  
  return model

In [117]:
#mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

Mostrando as informações da rede:

In [118]:
model = model_LSTM()
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_36 (LSTM)               (None, 20, 33)            5148      
_________________________________________________________________
lstm_37 (LSTM)               (None, 20, 33)            8844      
_________________________________________________________________
lstm_38 (LSTM)               (None, 20, 33)            8844      
_________________________________________________________________
lstm_39 (LSTM)               (None, 33)                8844      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 34        
Total params: 31,714
Trainable params: 31,714
Non-trainable params: 0
_________________________________________________________________
None


### Treinando a rede

In [119]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),  batch_size=100, epochs=200, verbose=1, callbacks=[es])

Epoch 1/200


TypeError: in user code:

    C:\Users\Edno\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Edno\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Edno\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Edno\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Edno\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C

In [ ]:
predicted_stock_price = model.predict(X_test) 

In [ ]:
predicted_stock_price

array([[-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793252e-32],
       [-8.8793252e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],
       [-8.8793205e-32],


## Visualização dos resultados

In [ ]:
#tp e tn são os corretos
# tn, fp, fn, tp=confusion_matrix(real,predito).ravel()
# acuracia=((tp+tn)/(tp+tn+fp+fn))*100
# print("n-----------------------Verdadeiro Positivo:{}\nVerdadeiro Negativo:{}\n-----------------------\nFalso Positivo:{}\nFalso Negativo:{}\n-----------------------".format(tp,tn,fp,fn))
# print("Acurracia:{:.2f}".format(acuracia))